# Flyte Demo

- Register all tasks
- Create an orchasteration workflow by composing other workflows
- Visualize the output

In [ ]:
# We will use our own staging environment for the live demo!
import os
os.environ['FLYTE_PLATFORM_URL'] = "flyte-staging.lyft.net"
version = "fake19"

In [ ]:
# Register existing tasks/workflows with the system
from flytekit.clis.sdk_in_container.register import register_all, register_tasks_only
import os 
from flytekit.configuration import set_flyte_config_file

os.environ["FLYTE_INTERNAL_IMAGE"] = "docker.io/lyft/flytekubecondemo2019:c139801bf4a8bc0d78c2bf56a30d03cca9205a64"
set_flyte_config_file("staging.config")
os.environ["FLYTE_INTERNAL_CONFIGURATION_PATH"] = "/app/staging.config"

register_all("flytekubecondemo2019", "development", ["workflows"], version=version, test=False)

# Orchestrator Workflow

- Use data preparation, training and evaluation workflows to compose an orchestrator worklfow
- Kick of an execution and wait for completion
- Visualize the output

In [ ]:
orchasterator_wf_version = version+"-orchasterator-7"

# The Orchestrator Workflow
import ujson
from flytekit.common.tasks.task import SdkTask
from flytekit.sdk.tasks import python_task, inputs, outputs
from flytekit.sdk.types import Types
from flytekit.sdk.workflow import workflow_class, Output, Input
from workflows.classifier_evaluate_workflow import evaluate_lp
from workflows.classifier_train_workflow import train_lp, DEFAULT_VALIDATION_DATA_RATIO, \
    DEFAULT_TRAINING_VALIDATION_CONFIG_FILE
from workflows.data_preparation_workflow import data_prep
from workflows.driver_workflow import pick_second

DEFAULT_TRAINING_CONFIG_FILE = "models/classifier/resnet50/configs/model_training_config_demo.json"
DEFAULT_EVALUATION_CONFIG_FILE = "models/classifier/resnet50/configs/model_evaluation_config_demo.json"


# Consume a task from a different project
# compute_confusion_matrix = SdkTask.fetch(
#     project="kubecondemo2019-metrics",
#     domain="development",
#     name="demo_metrics.tasks.confusion_matrix.confusion_matrix",
#     version="66b463748f25ef71c8cd4eb3001f00eafb83efc6",
# )


@workflow_class
class Orchestrator(object):
    # Define inputs
    streams_external_storage_prefix = Input(Types.String, required=True)
    streams_names = Input([Types.String], required=True)
    stream_extension = Input(Types.String, default="avi")

    streams_metadata_path = Input(Types.String, required=True)
    training_validation_config_json = Input(Types.Generic,
                                            default=ujson.loads(open(DEFAULT_TRAINING_CONFIG_FILE).read()))
    validation_data_ratio = Input(Types.Float, default=DEFAULT_VALIDATION_DATA_RATIO)
    evaluation_config_json = Input(Types.Generic,
                                   default=ujson.loads(open(DEFAULT_EVALUATION_CONFIG_FILE).read()))

    # Define workflow steps
    
    # Call data prep workflow defined in this project
    prepare = data_prep(
        streams_external_storage_prefix=streams_external_storage_prefix,
        streams_names=streams_names,
        stream_extension=stream_extension)

    # Call train workflow defined in this project
    train = train_lp(
        available_streams_names=prepare.outputs.streams_names_out,
        available_streams_mpblobs=prepare.outputs.selected_frames_mpblobs,
        streams_metadata_path=streams_metadata_path,
        training_validation_config_json=training_validation_config_json,
        validation_data_ratio=validation_data_ratio
    )

    # Call a task defined in this project
    pick_second = pick_second(models=train.outputs.trained_models)
    
    # Call evaluate workflow defined in this project
    evaluate = evaluate_lp(
        available_streams_names=prepare.outputs.streams_names_out,
        available_streams_mpblobs=prepare.outputs.selected_frames_mpblobs,
        streams_metadata_path=streams_metadata_path,
        evaluation_config_json=evaluation_config_json,
        model=pick_second.outputs.second,
        validation_data_ratio=1.0,
    )

    # Call compute_confusion_matrix task imported from a different project.
#     confusion_matrix_task = compute_confusion_matrix(
#         y_true=evaluate.outputs.ground_truths,
#         y_pred=evaluate.outputs.predictions,
#         title="Confusion Matrix",
#         normalize=True,
#         classes=["busy", "clear"],
#     )

    # Define workflow outputs
    model = Output(pick_second.outputs.second, sdk_type=Types.Blob)
    ground_truths = Output(evaluate.outputs.ground_truths, sdk_type=[Types.Integer])
    predictions = Output(evaluate.outputs.predictions, sdk_type=[Types.Integer])
#     confusion_matrix_image = Output(confusion_matrix_task.outputs.visual, sdk_type=Types.Blob)


orchestrator_lp = Orchestrator.create_launch_plan()

# Register the spec in the system
Orchestrator.register("flytekubecondemo2019", "development", "Orchestrator", orchasterator_wf_version)
orchestrator_lp.register("flytekubecondemo2019", "development", "Orchestrator", orchasterator_wf_version)

In [ ]:
# Kick of an execution
execution = orchestrator_lp.execute("flytekubecondemo2019", "development", inputs={
    'streams_external_storage_prefix': 's3://lyft-modelbuilder/metadata/_FlyteKubeconDemo2019Dataset/streams',
    'streams_names': ["1537396038_cam-rgb-1","1537396038_cam-rgb-2","1537396662_cam-rgb-1","1537396662_cam-rgb-2","1537396790_cam-rgb-1","1537396790_cam-rgb-2","1537396942_cam-rgb-1","1537396942_cam-rgb-2","1538521877_cam-rgb-1","1538521877_cam-rgb-2","1538521964_cam-rgb-1","1538521964_cam-rgb-2","1538522195_cam-rgb-1","1538522195_cam-rgb-2","1538522386_cam-rgb-1","1538522386_cam-rgb-2","1538522615_cam-rgb-1","1538522615_cam-rgb-2","1538522881_cam-rgb-1","1538522881_cam-rgb-2","1538523052_cam-rgb-1","1538523052_cam-rgb-2","1538523280_cam-rgb-1","1538523280_cam-rgb-2","1538523741_cam-rgb-1","1538523741_cam-rgb-2","1538523916_cam-rgb-1","1538523916_cam-rgb-2","1538524089_cam-rgb-1","1538524089_cam-rgb-2","1539390982_cam-rgb-1","1539390982_cam-rgb-2","1539391169_cam-rgb-1","1539391169_cam-rgb-2","1539391321_cam-rgb-1","1539391321_cam-rgb-2","1539391462_cam-rgb-1","1539391462_cam-rgb-2","1539391643_cam-rgb-1","1539391643_cam-rgb-2","1539391807_cam-rgb-1","1539391807_cam-rgb-2","1539391941_cam-rgb-1","1539391941_cam-rgb-2","1539471715_cam-rgb-1","1539471715_cam-rgb-2","1539471904_cam-rgb-1","1539471904_cam-rgb-2","1539475636_cam-rgb-1","1539475636_cam-rgb-2"],
    'streams_metadata_path': 's3://lyft-modelbuilder/metadata/_FlyteKubeconDemo2019Dataset/metadata/streams_metadata.json',
})

print("Started execution: {}".format(execution.id))

In [ ]:
from flytekit.common.workflow_execution import SdkWorkflowExecution
execution = SdkWorkflowExecution.fetch("flytekubecondemo2019", "development", "jnxxi2aone")

execution.wait_for_completion()
print("Workflow produced {} outputs".format(len(execution.outputs)))

# Download the model locally to run predictions
execution.outputs["model"].download("model", overwrite=True)
print("Downloaded the generated model.")

# Load the model
from keras.models import load_model
model = load_model("model")
print("Loaded the model. Ready to run predictions!")

In [ ]:
from IPython.display import Image

local_file = "/tmp/cm-4.png"
execution.outputs["confusion_matrix_image"].download(local_file, overwrite=True)
Image(filename=local_file)

In [ ]:
import os
import numpy as np
from keras.models import load_model
from keras.preprocessing import image
import matplotlib.pyplot as plt


def predict_and_render(imags_paths, model):
    imgs = [image.load_img(img_path, target_size=[224, 224]) for img_path in imgs_paths]
    xs = [image.img_to_array(img) for img in imgs]
    xs = [np.expand_dims(x, axis=0) for x in xs]
    xs = np.vstack(xs)
    preds = model.predict(np.array(xs))
    fig = plt.figure()
    
    fig, axes = plt.subplots(2, 5, figsize=(16,6))
    plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[]);
    
    for i, img in enumerate(imgs):
        ax_x = i // 5
        ax_y = i % 5
        for axis in ['top','bottom','left','right']:
            axes[ax_x, ax_y].spines[axis].set_linewidth(4.0)
        if preds[i][1] > 0.06739821285009384: 
            plt.setp(axes[ax_x, ax_y].spines.values(), color='red')
        else:
            plt.setp(axes[ax_x, ax_y].spines.values(), color='green')
        axes[ax_x, ax_y].imshow(imgs[i])


imgs_paths = [os.path.join("demo_imgs", img_path) for img_path in os.listdir("demo_imgs")]
predict_and_render(imgs_paths, model)
    